In [ ]:
# %pip install mediapipe opencv-python

In [10]:
import cv2
import mediapipe as mp
import numpy as np
from PIL import Image
from pprint import pprint
import pyttsx3

tts = pyttsx3.init()
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_drawing_styles = mp.solutions.drawing_styles

In [11]:
from pathlib import Path

img_path = Path(r"C:\Users\manoj\Downloads\handstand.png")
video_path = Path(r"C:\Users\manoj\Downloads\handstandgit.mp4")
long_video_path = Path(r"C:\Users\manoj\Downloads\handstandlong.mp4")
print(str(video_path))

C:\Users\manoj\Downloads\handstandgit.mp4


In [12]:
def get_angle(a: list[float], b: list[float], c: list[float]) -> float:
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(
        a[1] - b[1], a[0] - b[0]
    )
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle


def get_all_angles(modifiedlandmarks: dict[str, list[float]]) -> dict[str, float]:
    angles = {
        "LEFT_ELBOW": get_angle(
            modifiedlandmarks["LEFT_SHOULDER"],
            modifiedlandmarks["LEFT_ELBOW"],
            modifiedlandmarks["LEFT_WRIST"],
        ),
        "RIGHT_ELBOW": get_angle(
            modifiedlandmarks["RIGHT_SHOULDER"],
            modifiedlandmarks["RIGHT_ELBOW"],
            modifiedlandmarks["RIGHT_WRIST"],
        ),
        "LEFT_SHOULDER": get_angle(
            modifiedlandmarks["LEFT_HIP"],
            modifiedlandmarks["LEFT_SHOULDER"],
            modifiedlandmarks["LEFT_ELBOW"],
        ),
        "RIGHT_SHOULDER": get_angle(
            modifiedlandmarks["RIGHT_HIP"],
            modifiedlandmarks["RIGHT_SHOULDER"],
            modifiedlandmarks["RIGHT_ELBOW"],
        ),
        "LEFT_HIP": get_angle(
            modifiedlandmarks["LEFT_SHOULDER"],
            modifiedlandmarks["LEFT_HIP"],
            modifiedlandmarks["LEFT_KNEE"],
        ),
        "RIGHT_HIP": get_angle(
            modifiedlandmarks["RIGHT_SHOULDER"],
            modifiedlandmarks["RIGHT_HIP"],
            modifiedlandmarks["RIGHT_KNEE"],
        ),
        "LEFT_KNEE": get_angle(
            modifiedlandmarks["LEFT_HIP"],
            modifiedlandmarks["LEFT_KNEE"],
            modifiedlandmarks["LEFT_ANKLE"],
        ),
        "RIGHT_KNEE": get_angle(
            modifiedlandmarks["RIGHT_HIP"],
            modifiedlandmarks["RIGHT_KNEE"],
            modifiedlandmarks["RIGHT_ANKLE"],
        ),
    }
    return angles


def verify_angles(
    angles: dict[str, float], lower_angle: int = 150, upper_angle: int = 180
) -> tuple[bool, str | None]:
    for point in angles:
        if not (lower_angle <= angles[point] <= upper_angle):
            return False, point
    return True, None


def draw_landmarks(
    image, angles: dict[str, float], landmarks: dict[str, list[float]]
) -> None:
    for point in angles.keys():
        cv2.putText(
            image,
            f"{angles[point]:.2f}",
            tuple(
                np.multiply(landmarks[point], [image.shape[1], image.shape[0]]).astype(
                    int
                )
            ),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 255, 255),
            2,
            cv2.LINE_AA,
        ),


def write_to_file(file_name: str = "file.txt", content: str = "") -> None:
    with open(file_name, "w") as file:
        file.write(content)


def read_from_file(file_name: str = "file.txt") -> str:
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return ""


def get_important_landmarks(landmarks) -> dict[str, list[float]]:
    important_points = [
        "LEFT_WRIST",
        "RIGHT_WRIST",
        "LEFT_ELBOW",
        "RIGHT_ELBOW",
        "LEFT_SHOULDER",
        "RIGHT_SHOULDER",
        "LEFT_HIP",
        "RIGHT_HIP",
        "LEFT_KNEE",
        "RIGHT_KNEE",
        "LEFT_ANKLE",
        "RIGHT_ANKLE",
    ]
    return {
        point: [
            landmarks[mp_pose.PoseLandmark[point].value].x,
            landmarks[mp_pose.PoseLandmark[point].value].y,
        ]
        for point in important_points
    }

In [5]:
# VIDEO FEED
cap = cv2.VideoCapture(str(video_path))
# cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow("Mediapipe Feed", cv2.flip(frame, 1))

    if cv2.waitKey(10) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

In [6]:
IMAGE_FILES = [str(img_path)]
BG_COLOR = (192, 192, 192)  # gray
with mp_pose.Pose(
    static_image_mode=True,
    model_complexity=2,
    enable_segmentation=False,
    min_detection_confidence=0.5,
) as pose:
    for idx, file in enumerate(IMAGE_FILES):
        frame = cv2.imread(file)

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass

        modified_landmarks = get_important_landmarks(landmarks)
        angles = get_all_angles(modified_landmarks)
        draw_landmarks(image, angles, modified_landmarks)

        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
        )

        Image.fromarray(image).show()

In [5]:
cap = cv2.VideoCapture(str(long_video_path))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Total frames: {total_frames}, FPS: {fps}")

Total frames: 1907, FPS: 30.0


In [13]:
import threading
from time import sleep


def speak():
    tts = pyttsx3.init()
    tts.setProperty("rate", 150)  # Set speech rate
    while True:
        content = read_from_file(file_name="file.txt")
        if content:
            if tts._inLoop:
                tts.endLoop()
            tts.say(content)
            tts.runAndWait()
            write_to_file(file_name="file.txt", content="")
        sleep(2)


thread = threading.Thread(target=speak, daemon=True)

In [9]:
thread.join(1)

In [14]:
thread.is_alive()

False

In [15]:
cap = cv2.VideoCapture(str(long_video_path))
## Setup mediapipe instance
with mp_pose.Pose(
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6,
) as pose:
    thread.start()
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        # Flip the image horizontally for a later selfie-view display
        image = cv2.flip(frame, 1)

        # Recolor image to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass

        # Draw landmarks and angles
        modified_landmarks = get_important_landmarks(landmarks)
        angles = get_all_angles(modified_landmarks)
        draw_landmarks(image, angles, modified_landmarks)

        # Verify angles
        verify, point = verify_angles(angles, lower_angle=150, upper_angle=180)
        if not verify:
            write_to_file(file_name="file.txt", content=point.replace("_", " "))
        print(verify, point, angles[point] if not verify else None)

        # Render detections
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
        )

        cv2.imshow("Handstand", image)

        if cv2.waitKey(10) & 0xFF == ord("q"):
            write_to_file(file_name="file.txt", content="")
            break

    cap.release()
    cv2.destroyAllWindows()

False LEFT_HIP 100.02057704095775
False LEFT_HIP 99.01341164585155
False LEFT_HIP 98.04441568577343
False LEFT_HIP 96.31733966933496
False LEFT_HIP 95.97206464685793
False LEFT_HIP 94.36872540147054
False LEFT_SHOULDER 148.95400959893252
False LEFT_SHOULDER 147.6458463271407
False LEFT_SHOULDER 144.94340072093863
False LEFT_SHOULDER 143.29458165453772
False LEFT_SHOULDER 138.65636032905346
False LEFT_SHOULDER 133.60746238481687
False LEFT_SHOULDER 129.45145197738944
False LEFT_SHOULDER 125.73700890324335
False LEFT_SHOULDER 123.77768791102787
False LEFT_SHOULDER 119.56940363197835
False LEFT_SHOULDER 117.5957676972367
False LEFT_SHOULDER 111.8536254017499
False LEFT_ELBOW 148.7149366924111
False LEFT_ELBOW 147.99385325414767
False LEFT_ELBOW 148.98818525118898
False LEFT_SHOULDER 99.95319784937799
False LEFT_SHOULDER 103.45828259874716
False LEFT_SHOULDER 105.70421621795424
False LEFT_SHOULDER 110.12924232176522
False LEFT_SHOULDER 115.38271262205299
False LEFT_SHOULDER 118.70505001230

Exception in thread Thread-4 (speak):
Traceback (most recent call last):
  File "c:\Users\manoj\anaconda3\envs\ds\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Users\manoj\anaconda3\envs\ds\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\manoj\anaconda3\envs\ds\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\manoj\AppData\Local\Temp\ipykernel_157748\3474359796.py", line 14, in speak
  File "c:\Users\manoj\anaconda3\envs\ds\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started


True None None
True None None
False LEFT_SHOULDER 149.60566369262597
False LEFT_SHOULDER 149.24685817510004
False LEFT_SHOULDER 148.44486117494287
False LEFT_SHOULDER 147.48744540206303
False LEFT_SHOULDER 147.09859253333488
False LEFT_SHOULDER 146.89751147332535
False LEFT_SHOULDER 147.02909439348647
False LEFT_SHOULDER 147.0641236507498
False LEFT_SHOULDER 147.0520854450337
False LEFT_SHOULDER 147.0336756981324
False LEFT_SHOULDER 147.03148356875698
False LEFT_SHOULDER 146.89691773048276
False LEFT_SHOULDER 146.86215453078353
False LEFT_SHOULDER 146.62509950856685
False LEFT_SHOULDER 146.27704949735943
False LEFT_SHOULDER 146.15167602859995
False LEFT_SHOULDER 146.05700626000367
False LEFT_SHOULDER 146.08962191288683
False LEFT_SHOULDER 146.24974780796873
False LEFT_SHOULDER 146.31677100554225
False LEFT_SHOULDER 146.43063582663558
False LEFT_SHOULDER 146.59399894800967
False LEFT_SHOULDER 146.76203586989288
False LEFT_SHOULDER 147.29664274377592
False LEFT_SHOULDER 147.7029791905414

In [8]:
mp_drawing.DrawingSpec??

Init signature:
mp_drawing.DrawingSpec(
    color: Tuple[int, int, int] = (224, 224, 224),
    thickness: int = 2,
    circle_radius: int = 2,
) -> None
Docstring:      DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2)
Source:        
@dataclasses.dataclass
class DrawingSpec:
  # Color for drawing the annotation. Default to the white color.
  color: Tuple[int, int, int] = WHITE_COLOR
  # Thickness for drawing the annotation. Default to 2 pixels.
  thickness: int = 2
  # Circle radius. Default to 2 pixels.
  circle_radius: int = 2
File:           c:\users\manoj\anaconda3\envs\ds\lib\site-packages\mediapipe\python\solutions\drawing_utils.py
Type:           type
Subclasses:     

In [40]:
important_landmarks = get_important_landmarks(landmarks)
pprint(important_landmarks)

{'LEFT_ANKLE': [0.46831968426704407, 0.1444380283355713],
 'LEFT_ELBOW': [0.4429980516433716, 0.8268292546272278],
 'LEFT_HIP': [0.4685867130756378, 0.4892749488353729],
 'LEFT_KNEE': [0.4723130464553833, 0.31670740246772766],
 'LEFT_SHOULDER': [0.4757247567176819, 0.7233765125274658],
 'LEFT_WRIST': [0.4534723460674286, 0.9279065132141113],
 'RIGHT_ANKLE': [0.4735932946205139, 0.13535618782043457],
 'RIGHT_ELBOW': [0.4471559226512909, 0.8343253135681152],
 'RIGHT_HIP': [0.472617506980896, 0.4935796558856964],
 'RIGHT_KNEE': [0.474957138299942, 0.3151125907897949],
 'RIGHT_SHOULDER': [0.4791662096977234, 0.7252435088157654],
 'RIGHT_WRIST': [0.45731738209724426, 0.9484072923660278]}


In [41]:
angles = get_all_angles(landmarks=landmarks)
pprint(angles)

{'LEFT_ELBOW': 156.5292687461507,
 'LEFT_HIP': 177.01649968909143,
 'LEFT_KNEE': 177.43504620433345,
 'LEFT_SHOULDER': 160.69904226792767,
 'RIGHT_ELBOW': 158.5555700654588,
 'RIGHT_HIP': 177.6297053974541,
 'RIGHT_KNEE': 178.81421355048414,
 'RIGHT_SHOULDER': 162.0263487644425}


In [43]:
verify_angles(angles)

True